## simulation user lifetimes 

In [5]:
import sys
import pandas as pd
import time
import datetime

In [29]:
## warnings handler
import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [11]:
from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')
metrics_db = application.new_db_engine('nanohub_metrics')

salesforce = application.new_salesforce_engine()
db_s = salesforce

2021-08-02 16:30:40,079| WARNING | Could not read SSH configuration file: ~/.ssh/config


2021-08-02 16:30:40,079 - [WARNING] sshtunnel.SSHTunnelForwarder [sshtunnel._read_ssh_config:1032]: Could not read SSH configuration file: ~/.ssh/config
2021-08-02 16:30:40,079 - [WARNING] sshtunnel.SSHTunnelForwarder [sshtunnel._read_ssh_config:1032]: Could not read SSH configuration file: ~/.ssh/config


2021-08-02 16:30:40,086| INFO    | 0 keys loaded from agent


2021-08-02 16:30:40,086 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.get_agent_keys:1060]: 0 keys loaded from agent
2021-08-02 16:30:40,086 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.get_agent_keys:1060]: 0 keys loaded from agent


2021-08-02 16:30:40,092| INFO    | 0 key(s) loaded


...

2021-08-02 16:30:40,571 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.get_keys:1117]: 0 key(s) loaded
2021-08-02 16:30:40,571 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.get_keys:1117]: 0 key(s) loaded


2021-08-02 16:30:40,578| INFO    | Connecting to gateway: db2.nanohub.org:22 as user 'wang2506'


2021-08-02 16:30:40,578 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.__init__:978]: Connecting to gateway: db2.nanohub.org:22 as user 'wang2506'
2021-08-02 16:30:40,578 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.__init__:978]: Connecting to gateway: db2.nanohub.org:22 as user 'wang2506'
2021-08-02 16:30:41,098 - [INFO] nanoHUB.containers.dataaccess [connection.get_connection_for:78]: Started SSH Tunnel with db2.nanohub.org


2021-08-02 16:30:41,318| INFO    | Opening tunnel: 0.0.0.0:33153 <> 127.0.0.1:3306


2021-08-02 16:30:41,318 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel._serve_forever_wrapper:1433]: Opening tunnel: 0.0.0.0:33153 <> 127.0.0.1:3306
2021-08-02 16:30:41,318 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel._serve_forever_wrapper:1433]: Opening tunnel: 0.0.0.0:33153 <> 127.0.0.1:3306
Obtained Salesforce access token ...... True


In [12]:
## read in data
import os
cwd = os.getcwd()
clusters_df = pd.read_csv(cwd+'/manual_data_filters/clustered_users.csv')
selfs_df = pd.read_csv(cwd+'/manual_data_filters/self_study_users.csv')
researchers_df = pd.read_csv(cwd+'/manual_data_filters/researchers.csv')

In [13]:
clusters_df.head(5)

,Unnamed: 0,username
0,10813,0.yao.yuan
1,10817,008dilip
2,10821,01024461660a
3,10826,01luiyi
4,10828,0214moon


In [14]:
selfs_df.head(5)

,Unnamed: 0,username
0,0,-100005
1,1,-100024
2,2,-100026
3,3,-100032
4,4,-10006


In [15]:
researchers_df.head(5)

,Unnamed: 0,usernames
0,0,hspwong
1,1,onayfeh
2,2,accelrys
3,3,himadri
4,4,fossum


In [16]:
sql_query = "select min(datetime), max(datetime), user from toolstart group by user"

In [17]:
lifetimes_df = pd.read_sql_query(sql_query,metrics_db)

In [18]:
lifetimes_df.head(5)

,min(datetime),max(datetime),user
0,0000-00-00 00:00:00,2013-05-28 15:46:14,
1,2017-03-08 10:15:23,2017-03-09 05:09:22,0.yao.yuan
2,2016-09-28 20:27:13,2016-10-12 23:50:56,008dilip
3,2013-11-19 00:53:23,2013-11-19 00:58:56,00ff
4,2017-03-20 09:46:36,2017-03-20 09:46:36,00thamizharasi00


In [19]:
lifetimes_df.shape

(154748, 3)

In [20]:
# remove garbage usernames
temp = lifetimes_df[lifetimes_df['user'] != '']

In [21]:
temp

,min(datetime),max(datetime),user
1,2017-03-08 10:15:23,2017-03-09 05:09:22,0.yao.yuan
2,2016-09-28 20:27:13,2016-10-12 23:50:56,008dilip
3,2013-11-19 00:53:23,2013-11-19 00:58:56,00ff
4,2017-03-20 09:46:36,2017-03-20 09:46:36,00thamizharasi00
5,2020-09-23 18:14:20,2020-12-11 02:58:30,01024461660a
...,...,...,...
154743,2021-07-05 13:19:19,2021-07-05 13:27:28,zzzwmhq
154744,2002-07-23 02:52:49,2002-07-23 03:04:24,z_depth
154745,2006-10-16 20:25:49,2006-10-16 20:27:53,Z_E_U_S
154746,2013-02-11 14:42:41,2013-02-19 01:53:39,z_henry35


In [24]:
lifetimes_df['min(datetime)'].to_numpy()[0]

'0000-00-00 00:00:00'

In [22]:
lifetimes_df['min(datetime)'].to_list()[0] == '0000-00-00 00:00:00'

True

In [25]:
# replace 0000-00-00 with 2000-01-01
temp2 = lifetimes_df['min(datetime)'].to_list()
for i,j in enumerate(temp2):
    if type(j) == str:
        if j == '0000-00-00 00:00:00':
            temp2[i] = '2000-01-01 00:10:10'


In [26]:
len(temp2)

154748

In [27]:
temp2[0] == '0000-00-00 00:00:00'

False

In [30]:
temp['min(datetime)'] = temp2[1:]
temp['min(datetime)'] = pd.to_datetime(temp['min(datetime)'])
temp['max(datetime)'] = pd.to_datetime(temp['max(datetime)'])

In [31]:
## calculate lifetimes
lifetime_num = []
min_dates = temp['min(datetime)'].to_list()
for i,j in enumerate(temp['max(datetime)'].to_list()):
    temp_time = j-min_dates[i]
    lifetime_num.append(temp_time.days)

In [32]:
lifetime_num[:5]

[0, 14, 0, 0, 78]

In [33]:
temp['lifetimes'] = lifetime_num

In [34]:
display(temp.head(5))
display(temp.tail(5))

,min(datetime),max(datetime),user,lifetimes
1,2017-03-08 10:15:23,2017-03-09 05:09:22,0.yao.yuan,0
2,2016-09-28 20:27:13,2016-10-12 23:50:56,008dilip,14
3,2013-11-19 00:53:23,2013-11-19 00:58:56,00ff,0
4,2017-03-20 09:46:36,2017-03-20 09:46:36,00thamizharasi00,0
5,2020-09-23 18:14:20,2020-12-11 02:58:30,01024461660a,78


,min(datetime),max(datetime),user,lifetimes
154743,2021-07-05 13:19:19,2021-07-05 13:27:28,zzzwmhq,0
154744,2002-07-23 02:52:49,2002-07-23 03:04:24,z_depth,0
154745,2006-10-16 20:25:49,2006-10-16 20:27:53,Z_E_U_S,0
154746,2013-02-11 14:42:41,2013-02-19 01:53:39,z_henry35,7
154747,2008-07-09 22:51:51,2008-07-23 09:39:34,z_sath,13


In [35]:
from copy import deepcopy

In [36]:
send_df = deepcopy(temp)

In [37]:
send_df = send_df.rename(columns={'user':'nanoHUB_username__c','lifetimes':'sim_lifetimes__c',\
                    'min(datetime)':'earliest_sim__c','max(datetime)':'latest_sim__c'})

In [38]:
send_df

,earliest_sim__c,latest_sim__c,nanoHUB_username__c,sim_lifetimes__c
1,2017-03-08 10:15:23,2017-03-09 05:09:22,0.yao.yuan,0
2,2016-09-28 20:27:13,2016-10-12 23:50:56,008dilip,14
3,2013-11-19 00:53:23,2013-11-19 00:58:56,00ff,0
4,2017-03-20 09:46:36,2017-03-20 09:46:36,00thamizharasi00,0
5,2020-09-23 18:14:20,2020-12-11 02:58:30,01024461660a,78
...,...,...,...,...
154743,2021-07-05 13:19:19,2021-07-05 13:27:28,zzzwmhq,0
154744,2002-07-23 02:52:49,2002-07-23 03:04:24,z_depth,0
154745,2006-10-16 20:25:49,2006-10-16 20:27:53,Z_E_U_S,0
154746,2013-02-11 14:42:41,2013-02-19 01:53:39,z_henry35,7


In [42]:
print(send_df['nanoHUB_username__c'].to_list().index('drjohnbegg'))
print(send_df.iloc[37052,:]) ## a sanity check

37052
earliest_sim__c        2020-02-10 01:55:57
latest_sim__c          2021-08-01 09:56:56
nanoHUB_username__c             drjohnbegg
sim_lifetimes__c                       538
Name: 37053, dtype: object


In [43]:
## send to salesforce

In [44]:
db_s.external_id = external_id
db_s.object_id = object_id

In [45]:
db_s.send_data(send_df)

[Success] Bulk job creation successful. Job ID = 7505w00000YbviFAAR
hello
[Success] CSV upload successful. Job ID = 7505w00000YbviFAAR
[Success] Closing job successful. Job ID = 7505w00000YbviFAAR


In [53]:
db_s.check_bulk_status()

{'id': '7505w00000YbviFAAR',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-08-02T16:39:49.000+0000',
 'systemModstamp': '2021-08-02T16:46:47.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'nanoHUB_username__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 154747,
 'numberRecordsFailed': 729,
 'retries': 0,
 'totalProcessingTime': 1815069,
 'apiActiveProcessingTime': 1627889,
 'apexProcessingTime': 13111098}

In [54]:
import pprint
pprint.pprint(db_s.check_bulk_failed_results())

('"sf__Id","sf__Error",earliest_sim__c,latest_sim__c,nanoHUB_username__c,sim_lifetimes__c\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","2017-04-26","2017-04-26","1368872691zhangheng","0.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","2004-08-26","2004-09-26","30701144","31.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","2004-07-26","2004-09-19","3418303","54.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","2006-04-04","2006-04-05","4330","1.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","2000-02-28","2000-02-28","595nta","0.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","2017-06-07","2017-06-07","6r1d","0.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","2005-03-07","2005-08-13","